## Data Preparation for further analysis -Toronto Crime Analysis

### Link Toronto neighborhoods from MCI dataset to census tracts of census Canada from Statcanada

In [1]:
import pandas as pd
import requests
import json

In [2]:
# import the major crimes csv
mci_df = pd.read_csv('C:/Users/Melkamu/OneDrive - University of Kwazulu-Natal/Season7/YU/Course 1/Project 1/Melkamu/Dataset/From Toronto Police acessed on Feb 02_2020/MCI_2014_to_2018.csv')
mci_df.head(2)

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,ObjectId
0,-79.385193,43.659229,2349,GO-20149004286,2014-06-20T10:55:00.000Z,2014-06-20T13:20:00.000Z,Apartment,2130,210,Theft Over,...,171.0,Friday,10,Theft Over,D52,76,Bay Street Corridor (76),43.659229,-79.385193,2001
1,-79.425400,43.777592,2350,GO-20142411379,2014-07-02T00:20:00.000Z,2014-07-02T02:58:00.000Z,Outside,1457,100,Pointing A Firearm,...,183.0,Wednesday,0,Assault,D32,36,Newtonbrook West (36),43.777592,-79.425400,2002


In [3]:
mci_df['Hood_ID'].nunique()
# crimes data has 140 neighborhoods

140

In [4]:
len(mci_df)

167525

#### Crime data has 167525 incidents, each incident has a neighborhood ID
#### Data from Canada Census API must be fetched per geographical area, the smallest area you can pull is a census tract. 
#### Before we start fetching census data, let's get a list of the census tracts that we require

#### First, we need a way to convert between neighborhood ID and census tract. Found a CSV file with the conversion.

In [54]:
merger1= pd.read_csv('C:/Users/Melkamu/OneDrive - University of Kwazulu-Natal/Season7/YU/Course 1/Project 1/Melkamu/Dataset/From Toronto Police acessed on Feb 02_2020/Cleaning and preparation/CensusTractsProfileToronto.csv') 

In [55]:
merger1_df = pd.DataFrame(merger1)
merger1_df.head(3)

,"CT UID - Census Tract Unique Identifier\n(""CMA Code""+""0""+""Census Tract ID""),\nn=531","CT ID - Census Tract Identifier,\nn=531","Neighb ID, \n n=140","Neighbourhood Name,\n n=140","CPA ID,\nn=46","CPA Name \n(Community Planning Area),\n n=46","Sub-LHIN ID,\nn=14","Sub-LHIN Name, \nn=14","LHIN ID,\nn=5","LHIN Name, \nn=5"
0,5350001.0,1.0,70,South Riverdale,752,South Riverdale Greenwood,705,South East,7,Toronto Central
1,5350002.0,2.0,77,Waterfront Communities-The Island,733,Downtown Centre,703,South West,7,Toronto Central
2,5350003.0,3.0,85,South Parkdale,731,Parkdale Roncesvalles,703,South West,7,Toronto Central


In [56]:
merger1_df.columns

Index([' CT UID - Census Tract Unique Identifier\n("CMA Code"+"0"+"Census Tract ID"),\nn=531',
       'CT ID - Census Tract Identifier,\nn=531', 'Neighb ID, \n n=140',
       'Neighbourhood Name,\n n=140', 'CPA ID,\nn=46',
       'CPA Name \n(Community Planning Area),\n n=46', 'Sub-LHIN ID,\nn=14',
       'Sub-LHIN Name, \nn=14', 'LHIN ID,\nn=5', 'LHIN Name, \nn=5'],
      dtype='object')

In [64]:
# rename the columns we need
selected_merger1_df = merger1_df.rename(columns={' CT UID - Census Tract Unique Identifier\n("CMA Code"+"0"+"Census Tract ID"),\nn=531':'census_tract',
                             'Neighb ID, \n n=140':'Hood_ID',
                             'Neighbourhood Name,\n n=140': 'Neighbourhood'})
                                                 
#'CPA Name,\n n=46': 'CPA','Sub-LHIN Name,\n n=14': 'Sub_LHIN','LHIN Name,\n n=5': 'LHIN'})

In [69]:
# extract the columns we need
merger1_df = selected_merger1_df[['census_tract', 'Hood_ID', 'Neighbourhood']]

In [70]:
merger1_df.head(3)

,census_tract,Hood_ID,Neighbourhood
0,5350001.0,70,South Riverdale
1,5350002.0,77,Waterfront Communities-The Island
2,5350003.0,85,South Parkdale


In [71]:
merger1_df['Hood_ID'].nunique()
# converstion df also has 140 neighborhoods, yay!

140

In [72]:
merger1_df['census_tract'].nunique()
# they are divided into 531 census tracts

531

In [74]:
# lets do a quick test to make sure that the neighborhood id's are equivalent to the MCI id's
# we can test name of neighborhood for a given neighborhood id from each dataframe
merger1_df['Neighbourhood'][merger1_df['Hood_ID']==44]

288    Flemingdon Park
289    Flemingdon Park
290    Flemingdon Park
291    Flemingdon Park
Name: Neighbourhood, dtype: object

In [75]:
mci_df['Neighbourhood'][mci_df['Hood_ID']==44]

347       Flemingdon Park (44)
475       Flemingdon Park (44)
524       Flemingdon Park (44)
578       Flemingdon Park (44)
676       Flemingdon Park (44)
                  ...         
165771    Flemingdon Park (44)
166519    Flemingdon Park (44)
166783    Flemingdon Park (44)
166845    Flemingdon Park (44)
166888    Flemingdon Park (44)
Name: Neighbourhood, Length: 1001, dtype: object

### we won't join these dataframes yet because each neighborhood has several tracts (and we can't average tract ids)
### so instead, we'll first get the metrics (eg. average income) for each cansus tracts, so that we can then average the metrics across tracts  per neighborhood

#### We want to fetch the census data from stat canada api

In [76]:
tracts_data = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CR2016Geo.json?lang=E&geos=CT&cpt=35')
# where CT indicates census tracts, and 35 indicates ontario

In [ ]:
# won't convert to json nicely, look at data
#tracts_data.text

In [96]:
tracts_text = tracts_data.text.replace('//','')
#tracts_text

In [97]:
tracts_data = json.loads(tracts_text)
#pprint(tracts_data)

In [98]:
# to df
tracts_df = pd.DataFrame.from_dict(tracts_data['DATA'])
tracts_df.columns=tracts_data['COLUMNS']

In [99]:
# extract columns we need - only dguid and tract id
tracts_df = tracts_df[['GEO_UID', 'GEO_ID_CODE']]
tracts_df.head(3)

,GEO_UID,GEO_ID_CODE
0,2016S05075050001.01,5050001.01
1,2016S05075050001.04,5050001.04
2,2016S05075050001.05,5050001.05


In [100]:
len(tracts_df)
# 2376 tracts in ontario, no need to use all

2376

#### Now we have the dguid for each tract code
#### but this is for all of ontario, lets join with the neighborhood conversion df so that we only keep toronto ones

In [101]:
# rename the tract column to be similar to the conversion df so that we can join them
tracts_df = tracts_df.rename(columns={'GEO_ID_CODE':'census_tract'})

In [ ]:
#toronto_tracts = merger1_df.join(tracts_df.set_index('census_tract'), on='census_tract')

# error due to joining on different data types

In [105]:
tracts_df.head(1)

,GEO_UID,census_tract
0,2016S05075050001.01,5050001.01


In [106]:
merger1_df.head(1)

,census_tract,Hood_ID,Neighbourhood
0,5350001.0,70,South Riverdale


In [111]:
type(tracts_df['census_tract'][1])

str

In [112]:
type(merger1_df['census_tract'][0])

numpy.float64

In [29]:
# one is str and one is float

In [113]:
# cast to str and try joining again
merger1_df['census_tract'] = merger1_df['census_tract'].map("{0:.2f}".format)

C:\Users\Melkamu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [114]:
toronto_tracts = merger1_df.join(tracts_df.set_index('census_tract'), on='census_tract', how='inner')
toronto_tracts.head(3)

,census_tract,Hood_ID,Neighbourhood,GEO_UID
0,5350001.00,70,South Riverdale,2016S05075350001.00
1,5350002.00,77,Waterfront Communities-The Island,2016S05075350002.00
2,5350003.00,85,South Parkdale,2016S05075350003.00


In [115]:
##DGUID-Dissemination Geography Unique Identifier
toronto_tracts = toronto_tracts.rename(columns={'GEO_UID':'DGUID'})

In [116]:
toronto_tracts.head(3)

,census_tract,Hood_ID,Neighbourhood,DGUID
0,5350001.00,70,South Riverdale,2016S05075350001.00
1,5350002.00,77,Waterfront Communities-The Island,2016S05075350002.00
2,5350003.00,85,South Parkdale,2016S05075350003.00


In [117]:
len(toronto_tracts)
# 494 dguids (down from 531, some were not in the census tracts data, possibly due to change of tract codes over years)

494

In [119]:
# output to csv
toronto_tracts.to_csv('C:/Users/Melkamu/OneDrive - University of Kwazulu-Natal/Season7/YU/Course 1/Project 1/Melkamu/Dataset/From Toronto Police acessed on Feb 02_2020/Cleaning and preparation/toronto_tracts.csv', index=False)

In [38]:
# conversion is ready to be used by analysis notebooks